In [2]:
import re

with open(r'cheater_PGNs\both_cheaters_PGN.txt') as f:
    pgn = f.read()

def moves_to_list(moves):
    moves_list = []
    for element in moves.split():
        if re.match('[a-zA-Z]+[1-9]|O-O|O-O-O', element) is not None:
            element = element.replace('?', '')
            element = element.replace('!', '')
            element = element.replace('+', '')

            moves_list.append(element)
    
    return moves_list

moves = moves_to_list(pgn)

In [3]:
len(moves)

47

In [4]:
import chess
import numpy as np

def save_board_state_after_move(moves):
    board = chess.Board()
    FEN_list = []
    try:
        for idx, move in enumerate(moves):
            board.push_san(move)
            FEN_list.append(board.fen())
        return FEN_list
    except Exception as e:
        print(e)
        print(board)
        return np.nan

fens = save_board_state_after_move(moves)

In [5]:
import pandas as pd
import numpy as np

next_fens = fens[1:]
next_fens.append(np.nan)

data = {'fens': fens, 'next_fens': next_fens}

input_data = pd.DataFrame(data=data)
input_data.dropna(inplace=True)

In [6]:
piece_to_token   = {'r' :        1,
                   'n' :        2,
                   'b' :        3,
                   'q' :        4,
                   'k' :        5,
                   'p' :        6,
                   'R' :        7,
                   'N' :        8,
                   'B' :        9,
                   'Q' :        10,
                   'K' :        11,
                   'P' :        12,
                   'empty' :    0}

def fen_to_network_input_tokenized(fen):
    encoded_board = []
    fen = fen.split()[0]
    for char in fen:
        if char in piece_to_token.keys():
            encoded_board.append(piece_to_token[char])
        elif char != '/':
            for _ in range(int(char)):
                encoded_board.append(piece_to_token['empty'])
    return encoded_board

In [7]:
input_data['FEN_tokenized'] = input_data['fens'].apply(fen_to_network_input_tokenized)
input_data['next_FEN_tokenized'] = input_data['next_fens'].apply(fen_to_network_input_tokenized)


In [8]:
input_data['input'] = input_data['FEN_tokenized'] + input_data['next_FEN_tokenized']

In [9]:
import torch

X = torch.tensor(input_data['input'],dtype=torch.int)

In [10]:
from model_lstm import CheaterClassifier

model = CheaterClassifier()

model.load_state_dict(torch.load('model_dospermiony.pt'))

<All keys matched successfully>

In [11]:
X.shape

torch.Size([46, 128])

In [12]:
model.eval()
outputs = model(X)

In [13]:
counter_white=0
counter_black=0
mean_proba_white = 0
mean_proba_black = 0
white = []
black = []
for idx, output in enumerate(outputs):
    if idx % 2 == 0:
        mean_proba_white += output
    else:
        mean_proba_black += output
    if output >= .5:
        if idx % 2 == 0:
            white.append((idx, output))
            counter_white+=1
        else:
            black.append((idx, output))
            counter_black+=1

mean_proba_white = mean_proba_white/len(outputs)
mean_proba_black = mean_proba_black/len(outputs)
print('Mean proba white: ', mean_proba_white)

print('Mean proba black: ', mean_proba_black)
print('white', counter_white)
print('black', counter_black)
print('##########white############')
for w in white:
    print(w)
print('##########black############')
for b in black:
    print(b)

Mean proba white:  tensor([0.3593], grad_fn=<DivBackward0>)
Mean proba black:  tensor([0.3644], grad_fn=<DivBackward0>)
white 16
black 16
##########white############
(14, tensor([0.5529], grad_fn=<UnbindBackward0>))
(16, tensor([0.7080], grad_fn=<UnbindBackward0>))
(18, tensor([0.7973], grad_fn=<UnbindBackward0>))
(20, tensor([0.8425], grad_fn=<UnbindBackward0>))
(22, tensor([0.8673], grad_fn=<UnbindBackward0>))
(24, tensor([0.8841], grad_fn=<UnbindBackward0>))
(26, tensor([0.8982], grad_fn=<UnbindBackward0>))
(28, tensor([0.9141], grad_fn=<UnbindBackward0>))
(30, tensor([0.9326], grad_fn=<UnbindBackward0>))
(32, tensor([0.9482], grad_fn=<UnbindBackward0>))
(34, tensor([0.9580], grad_fn=<UnbindBackward0>))
(36, tensor([0.9633], grad_fn=<UnbindBackward0>))
(38, tensor([0.9660], grad_fn=<UnbindBackward0>))
(40, tensor([0.9670], grad_fn=<UnbindBackward0>))
(42, tensor([0.9670], grad_fn=<UnbindBackward0>))
(44, tensor([0.9659], grad_fn=<UnbindBackward0>))
##########black############
(15, t